<a href="https://colab.research.google.com/github/ElenaGordienko1/NLP/blob/main/NLP5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Выпонение лабораторной работы 5

Импорт библиотек ⭐

In [7]:
!pip install transformers datasets scikit-learn --quiet

import pandas as pd
import numpy as np
import re
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score

In [8]:
pip install -U transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 114.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


###  Подготовка данных

In [9]:
df = pd.read_csv("/content/final_review_dataset_extended.csv")
df = df[['review', 'rating_value']].dropna()
df = df[df['review'].str.strip().astype(bool)]
df = df.rename(columns={'review': 'text', 'rating_value': 'label'})

df = df.iloc[:500].reset_index(drop=True)

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])
id2label = {i: label for i, label in enumerate(label_encoder.classes_)}
label2id = {label: i for i, label in id2label.items()}
num_classes = len(label_encoder.classes_)

X = df['text']
y = df['label']
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, stratify=y_train_val, random_state=42)

train_dataset = Dataset.from_dict({"text": X_train.tolist(), "label": y_train.tolist()})
val_dataset = Dataset.from_dict({"text": X_val.tolist(), "label": y_val.tolist()})
test_dataset = Dataset.from_dict({"text": X_test.tolist(), "label": y_test.tolist()})


### Токенизация

In [10]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length")

tokenized_train = train_dataset.map(tokenize, batched=True)
tokenized_val = val_dataset.map(tokenize, batched=True)
tokenized_test = test_dataset.map(tokenize, batched=True)

num_labels = int(len(label_encoder.classes_))
id2label = {int(i): str(label) for i, label in enumerate(label_encoder.classes_)}
label2id = {str(label): int(i) for i, label in enumerate(label_encoder.classes_)}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Обучение модели:

In [11]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=False,
    report_to="none",
    save_total_limit=1,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics
)

trainer.train()

test_results = trainer.evaluate(tokenized_test)

print("Результаты на тестовой выборке:")
print(f"Loss: {test_results['eval_loss']:.3f}")
print(f"Accuracy: {test_results['eval_accuracy']:.3f}")
print(f"F1-Score: {test_results['eval_f1']:.3f}")


<ipython-input-11-cbf57063b2bf>:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


Результаты на тестовой выборке:
Loss: 0.963
Accuracy: 0.710
F1-Score: 0.590


###  Тестирование и вывод

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

sample_text = X_test.iloc[0]
true_label_encoded = y_test.iloc[0]
true_label = label_encoder.inverse_transform([true_label_encoded])[0]

inputs = tokenizer(
    sample_text,
    return_tensors="pt",
    truncation=True,
    padding=True,
    max_length=256
).to(device)

with torch.no_grad():
    outputs = model(**inputs)

predicted_class_id = torch.argmax(outputs.logits, dim=1).item()
predicted_label = label_encoder.inverse_transform([predicted_class_id])[0]

# Печатаем результат
print(f"Текст отзыва: {sample_text}")
print(f"Прогнозируемая категория: {predicted_label}")
print(f"Истинная категория: {true_label}")


Текст отзыва: Банк самостоятельно открывает кредитные карты. У меня их было две, оказывается. Если просить их закрыть, это сделать не просто. Закрытие останавливается, если не проследить то она так и останется. Все отзывы, особенно те, где говорят о человеческом отношении и тем более к пожилым людям не вызывают доверие. Мне нужно было потратить несколько дней, что бы проконтролировать закрытие ненужной мне кредитки. И это еще не конец. Через месяц посмотрим
Прогнозируемая категория: 1
Истинная категория: 1


**Вывод по метрикам на тестовой выборке:**

Loss (потери) составил 0.961, что указывает на умеренную степень ошибки модели при прогнозировании.


Accuracy (точность) равна 71.0%, что означает, что модель правильно классифицировала примерно 7 из 10 отзывов.


F1-Score — 0.590 — говорит о том, что у модели средняя производительность.


**Заключение:** Модель угадала категорию отзывов, и по показателям метрик вроде все норм, средненько, но вероятно это из-за маленького количества данных.